> Import dependencies for Hand-on 1

In [1]:
import pandas as pd
import string
import requests
from bs4 import BeautifulSoup
import numpy as np

> Read and clean `software_development_usa.csv` dataset

In [2]:
def get_and_clean_data() -> pd.DataFrame:
    data = pd.read_csv("../../data/software_development_usa.csv")
    description = data["job_description"]
    cleaned_description = description.apply(
        lambda s: s.translate(str.maketrans("", "", string.punctuation + "\xa0"))
    )
    cleaned_description = cleaned_description.apply(lambda s: s.lower())
    cleaned_description = cleaned_description.apply(
        lambda s: s.translate(
            str.maketrans(string.whitespace, " " * len(string.whitespace), "")
        )
    )
    cleaned_description = cleaned_description.drop_duplicates()
    return cleaned_description

In [3]:
data = get_and_clean_data()
data.head()

0    the chosen sr software developer will be part ...
1    position c lead software developer location mi...
2    senior software developer hoboken nj starts as...
3    our client a multinational publishing and educ...
4    position c lead software developer location ph...
Name: job_description, dtype: object

> Tokenize the descriptions

In [4]:
def simple_tokenize(data: pd.DataFrame) -> pd.DataFrame:
    cleaned_description = data.apply(lambda s: [x.strip() for x in s.split()])
    return cleaned_description

In [5]:
data = simple_tokenize(data)
data.head()

0    [the, chosen, sr, software, developer, will, b...
1    [position, c, lead, software, developer, locat...
2    [senior, software, developer, hoboken, nj, sta...
3    [our, client, a, multinational, publishing, an...
4    [position, c, lead, software, developer, locat...
Name: job_description, dtype: object

> Combine `get_and_clean_data()` and `simple_tokenize()` into `parse_job_description()`

In [6]:
def parse_job_description():
    cleaned_description = get_and_clean_data()
    cleaned_description = simple_tokenize(cleaned_description)
    return cleaned_description

> Count `Python + MySQL`

In [7]:
def count_python_mysql():
    parsed_description = parse_job_description()
    count_python = parsed_description.apply(lambda s: "python" in s).sum()
    count_mysql = parsed_description.apply(lambda s: "mysql" in s).sum()
    print("python: " + str(count_python) + " of " + str(parsed_description.shape[0]))
    print("mysql: " + str(count_mysql) + " of " + str(parsed_description.shape[0]))


count_python_mysql()

python: 1379 of 7583
mysql: 667 of 7583


In [8]:
def parse_db() -> list[list[str]]:
    html_doc = requests.get("https://db-engines.com/en/ranking").content
    soup = BeautifulSoup(html_doc, "html.parser")
    db_table = soup.find("table", {"class": "dbi"})
    all_db = [
        "".join(s.find("a").findAll(text=True, recursive=True)).strip()
        for s in db_table.findAll("th", {"class": "pad-l"})
    ]
    all_db = list(dict.fromkeys(all_db))
    db_list = all_db[:10]
    db_list = [s.lower() for s in db_list]
    db_list = [[[x.strip() for x in s.split()][0]] for s in db_list]
    return db_list

In [9]:
parse_db()

/tmp/ipykernel_66464/4149286022.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "".join(s.find("a").findAll(text=True, recursive=True)).strip()


[['oracle'],
 ['mysql'],
 ['microsoft'],
 ['postgresql'],
 ['mongodb'],
 ['redis'],
 ['elasticsearch'],
 ['ibm'],
 ['sqlite'],
 ['microsoft']]

> How many database is in description?

In [10]:
cleaned_db = parse_db()
parsed_description = parse_job_description()
raw = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    raw[i] = parsed_description.apply(lambda s: np.all([x in s for x in db])).sum()
    print(" ".join(db) + ": " + str(raw[i]) + " of " + str(parsed_description.shape[0]))

/tmp/ipykernel_66464/4149286022.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "".join(s.find("a").findAll(text=True, recursive=True)).strip()


oracle: 1392 of 7583
mysql: 667 of 7583
microsoft: 1516 of 7583
postgresql: 261 of 7583
mongodb: 296 of 7583
redis: 106 of 7583
elasticsearch: 161 of 7583
ibm: 227 of 7583
sqlite: 28 of 7583
microsoft: 1516 of 7583


> How many of each database that alongside python?

In [11]:
with_oracle: list[pd.DataFrame] = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    with_oracle[i] = parsed_description.apply(
        lambda s: np.all([x in s for x in db]) and "python" in s
    ).sum()

    print(
        " ".join(db)
        + " + python: "
        + str(with_oracle[i])
        + " of "
        + str(parsed_description.shape[0])
    )

oracle + python: 243 of 7583
mysql + python: 207 of 7583
microsoft + python: 138 of 7583
postgresql + python: 90 of 7583
mongodb + python: 111 of 7583
redis + python: 38 of 7583
elasticsearch + python: 73 of 7583
ibm + python: 63 of 7583
sqlite + python: 7 of 7583
microsoft + python: 138 of 7583


In [12]:
for i, db in enumerate(cleaned_db):
    print(
        " ".join(db)
        + " + python "
        + str(with_oracle[i])
        + " of "
        + str(raw[i])
        + " ("
        + str(np.around(with_oracle[i] / raw[i] * 100, 2))
        + "%)"
    )

oracle + python 243 of 1392 (17.46%)
mysql + python 207 of 667 (31.03%)
microsoft + python 138 of 1516 (9.1%)
postgresql + python 90 of 261 (34.48%)
mongodb + python 111 of 296 (37.5%)
redis + python 38 of 106 (35.85%)
elasticsearch + python 73 of 161 (45.34%)
ibm + python 63 of 227 (27.75%)
sqlite + python 7 of 28 (25.0%)
microsoft + python 138 of 1516 (9.1%)


#### Try to indexing

In [13]:
langs = [
    ["java"],
    ["python"],
    ["c"],
    ["kotlin"],
    ["swift"],
    ["rust"],
    ["ruby"],
    ["scala"],
    ["julia"],
    ["lua"],
]
parsed_description = parse_job_description()
parsed_db = parse_db()
all_terms = langs + parsed_db
query_map = pd.DataFrame(
    parsed_description.apply(
        lambda s: [1 if np.all([d in s for d in db]) else 0 for db in all_terms]
    ).values.tolist(),
    columns=[" ".join(d) for d in all_terms],
)
query_map

/tmp/ipykernel_66464/4149286022.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  "".join(s.find("a").findAll(text=True, recursive=True)).strip()


,java,python,c,kotlin,swift,rust,ruby,scala,julia,lua,oracle,mysql,microsoft,postgresql,mongodb,redis,elasticsearch,ibm,sqlite,microsoft
0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7578,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
7579,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7580,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
7581,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0


### Challenge
1. What DB should I learn after java?
2. Which DB is in demand alongside oracle?
3. What programming language is in demand alongside python?

##### What DB should I learn after Java?
Oracle is the database that you should learn after Java.

In [14]:
with_oracle: list[pd.DataFrame] = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    with_oracle[i] = parsed_description.apply(
        lambda s: np.all([x in s for x in db]) and "java" in s
    ).sum()

    print(
        " ".join(db)
        + " + java: "
        + str(with_oracle[i])
        + " of "
        + str(parsed_description.shape[0])
    )

oracle + java: 913 of 7583
mysql + java: 397 of 7583
microsoft + java: 448 of 7583
postgresql + java: 161 of 7583
mongodb + java: 166 of 7583
redis + java: 40 of 7583
elasticsearch + java: 112 of 7583
ibm + java: 135 of 7583
sqlite + java: 5 of 7583
microsoft + java: 448 of 7583


##### Which DB is in demand alongside oracle?
MySQL is the DB that you should learn alongside oracle.

In [20]:
with_oracle: list[pd.DataFrame] = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    if db[0] == "oracle":
        continue

    with_oracle[i] = parsed_description.apply(
        lambda s: np.all([x in s for x in db]) and "oracle" in s
    ).sum()

    print(
        " ".join(db)
        + " + oracle: "
        + str(with_oracle[i])
        + " of "
        + str(parsed_description.shape[0])
    )

mysql + oracle: 312 of 7583
microsoft + oracle: 282 of 7583
postgresql + oracle: 100 of 7583
mongodb + oracle: 104 of 7583
redis + oracle: 12 of 7583
elasticsearch + oracle: 32 of 7583
ibm + oracle: 84 of 7583
sqlite + oracle: 17 of 7583
microsoft + oracle: 282 of 7583


#### What programming language is in demand alongside python?
Java is the programming language is in demand alongside python.

In [16]:
with_python: list[pd.DataFrame] = [None] * len(langs)
for i, db in enumerate(langs):
    if db[0] == "python":
        continue

    with_oracle[i] = parsed_description.apply(
        lambda s: np.all([x in s for x in db]) and "python" in s
    ).sum()

    print(
        " ".join(db)
        + " + python: "
        + str(with_oracle[i])
        + " of "
        + str(parsed_description.shape[0])
    )

java + python: 830 of 7583
c + python: 689 of 7583
kotlin + python: 6 of 7583
swift + python: 37 of 7583
rust + python: 6 of 7583
ruby + python: 181 of 7583
scala + python: 76 of 7583
julia + python: 1 of 7583
lua + python: 11 of 7583


### Tokenizer

In [17]:
str1 = "the chosen software developer will be part of a larger engineering team developing software for medical devices."

str2 = "we are seeking a seasoned software developer with strong analytical and technical skills to join our public sector technology consulting team."

In [18]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

/home/tkthanatorn/miniconda3/envs/quant/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tkthanatorn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tkthanatorn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
def prepare_string(data: str):
    tokened = word_tokenize(data)
    tokened = [w for w in tokened if len(w) > 2]
    clean_stopword = [word for word in tokened if not word in stopwords.words()]

    ps = PorterStemmer()
    stemmed = np.unique([ps.stem(w) for w in clean_stopword])
    return stemmed

prepared1 = prepare_string(str1)
prepared2 = prepare_string(str2)
full_list = np.sort(np.concatenate([prepared1, prepared2]))
full_list

array(['analyt', 'chosen', 'consult', 'develop', 'develop', 'devic',
       'engin', 'join', 'larger', 'medic', 'part', 'public', 'season',
       'sector', 'seek', 'skill', 'softwar', 'softwar', 'strong', 'team',
       'team', 'technic', 'technolog'], dtype='<U9')